<a href="https://colab.research.google.com/github/nastyatrvl/3D-plots/blob/master/matching_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir('/content/drive/MyDrive/ILR')

Mounted at /content/drive/


In [ ]:
!pip install jaro-winkler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install bertopic[visualization]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 3.6 MB/s 
     |████████████████████████████████| 88 kB 8.1 MB/s 
     |████████████████████████████████| 5.2 MB 40.7 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 85 kB 6.6 MB/s 
     |████████████████████████████████| 4.4 MB 36.1 MB/s 
     |████████████████████████████████| 1.2 MB 36.8 MB/s 
     |████████████████████████████████| 101 kB 13.1 MB/s 
     |████████████████████████████████| 636 kB 42.7 MB/s 
     |████████████████████████████████| 6.6 MB 49.2 MB/s 
     |████████████████████████████████| 1.1 MB 43.0 MB/s 
  Created wheel for hdbscan: filename=hdbscan-0.8.28-cp37-cp37m-linux_x86_64.whl size=2342148 sha256=ac0c896e65b77c327699e45f74ac58716c4c0aaabb60f59c248af5c4fbe8cca0
  Stored in directory: /root/.cache

In [ ]:
# essentials
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt
# text data essentials
import re
import string
# tfidf
from sklearn.feature_extraction.text import TfidfVectorizer
# jaro
import jaro
# parallel computing
from numba import prange
# nltk
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# Topic modelling
from bertopic import BERTopic
# Charts
import base64
from IPython.display import Image, display

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Helper functons

In [ ]:
def clean_text(string):
  try:
    temp = string.lower().translate(str.maketrans(symbols, ' ' * len(symbols))).replace("'", " ")
    temp = " ".join([w for w in temp.split() if (w not in stopwords) & (not w.isdigit())])
    return temp
  except AttributeError:
    return string

In [ ]:
def search_level(row, col_name): 
  string = row[col_name]
  split_w = string.lower().split()
  inter_w = list(set(split_w) & set(nummerals))
  if len(split_w)>1 and len(inter_w) > 0:
    return pd.Series([inter_w[0], ' '.join([w for w in split_w if w!=inter_w[0]])])
  else:
    return pd.Series(['', string.lower()])

In [ ]:
def determine_seniority(title, seniority_list):
  return list(set(title.split()) & set(seniority_list))

In [ ]:
def remove_all_indicators(title):
  all_indicators = senior + junior + manager + engineer
  return " ".join([w for w in title.lower().split() if w not in all_indicators])

In [ ]:
def stem_title(row):
  ps = PorterStemmer()
  title = row['job_title_clean_no_indicators']
  stemmed_title = [ps.stem(w) for w in title.split()]
  stemmed_title_j = ' '.join(stemmed_title)
  return pd.Series([stemmed_title_j, stemmed_title])

In [ ]:
def seniority_col(row, col1, col2):
  if len(row[col1]) > 0:
    return col1
  elif len(row[col2]) > 0:
    return col2
  else:
    return 'none'

In [ ]:
def preprocess(df, col_name):
  df[["level", "job_title_wt_level"]] = df.apply(lambda x: search_level(x, col_name), axis=1, result_type='expand')
  df['job_title_orig_clean'] = df[col_name].apply(lambda x: clean_text(x))
  df['job_title_clean'] = df["job_title_wt_level"].apply(lambda x: clean_text(x))
  # seniority + role
  df['senior'] = df["job_title_clean"].apply(lambda x: determine_seniority(x, senior))
  df['junior'] = df["job_title_clean"].apply(lambda x: determine_seniority(x, junior))
  df['manager'] = df["job_title_clean"].apply(lambda x: determine_seniority(x, manager))
  df['engineer'] = df["job_title_clean"].apply(lambda x: determine_seniority(x, engineer))
  df['job_title_clean_no_indicators'] = df['job_title_clean'].apply(lambda x: remove_all_indicators(x))
  df[["stemmed_title_j", "stemmed_title"]] = df.apply(lambda x: stem_title(x), axis=1, result_type='expand')
  df['m_e'] = df.apply(lambda x: seniority_col(x, 'manager', 'engineer'), axis=1)
  df['s_j'] = df.apply(lambda x: seniority_col(x, 'senior', 'junior'), axis=1)
  df = df[df['job_title_clean_no_indicators']!=''].dropna(subset=['job_title_clean_no_indicators'])
  df['first_letter'] = df['job_title_clean_no_indicators'].apply(lambda x: x[0])
  return df

## Preprocessing

In [ ]:
salarydot = pd.read_csv('data/salary_dot_com_data_yearly.csv')
payscale = pd.read_csv('data/payscale_data_yearly.csv')
glassdoor = pd.read_csv('data/glassdoor_data_yearly.csv')
indeed = pd.read_csv('data/indeed_data_yearly.csv')
onet = pd.read_excel('data/onet_alternate.xlsx')
onet_orig = pd.read_excel('data/onet.xlsx')

In [ ]:
onet_orig.columns = ['soc_code', 'title', 'description']
onet_orig = onet_orig[['title', 'soc_code']]

In [ ]:
onet.columns = ['soc_code', 'title', 'alternate_title', 'short_title', 'source']
onet = onet[['title', 'alternate_title', 'soc_code']]

In [ ]:
# no duplicates here
print(len(salarydot))
print(len(payscale))
print(len(glassdoor))
print(len(indeed))
print(len(onet))
print(len(onet_orig))

14512
12335
894
11947
60511
1016


In [ ]:
indeed['title_detailed'] = [i.split('career/')[1][:-9].replace('-', ' ') for i in indeed['url']]
salarydot['job_title_clean'] = [i.replace('&amp;', 'and').replace('&#39;s', '') for i in salarydot['job_title'] if not i.isdigit()]
indeed['job_title_clean'] = [i.replace(' amp ', ' and ') for i in indeed['title_detailed']]

In [ ]:
nummerals = ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x']
symbols = "!\"#$%&()*+-./:;,<=>?@[\]^_`{|}~\n"
stopwords = stopwords.words("english")
stopwords = [i for i in stopwords if i not in nummerals]
senior = ['senior', 'sr', 'director', 'executive', 'vice', 'president', 'chief', 'head', 'lead', 'top']
junior = ['junior', 'jr', 'entry', 'intern', 'trainee']
manager = ['manager', 'supervisor']
engineer = ['engineer', 'developer']

In [ ]:
salarydot = preprocess(salarydot, 'job_title_clean')
payscale = preprocess(payscale, 'title')
indeed = preprocess(indeed, 'job_title_clean')
glassdoor = preprocess(glassdoor, 'title')
onet = preprocess(onet, 'alternate_title')
onet_orig = preprocess(onet_orig, 'title')

## Topic modelling

In [ ]:
stemmed_titles_j = list(salarydot['stemmed_title_j']) + \
                   list(payscale['stemmed_title_j']) + \
                   list(indeed['stemmed_title_j']) + \
                   list(glassdoor['stemmed_title_j']) + \
                   list(onet['stemmed_title_j']) + \
                   list(onet_orig['stemmed_title_j'])

In [ ]:
topic_model = BERTopic(
                 top_n_words = 20,
                 nr_topics = 5,
                 n_gram_range = (1, 2),
                 min_topic_size = 10,
                 calculate_probabilities=True,
                 verbose=True)

In [ ]:
topics, probs = topic_model.fit_transform(stemmed_titles_j)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/3159 [00:00<?, ?it/s]

2022-07-15 03:05:25,649 - BERTopic - Transformed documents to Embeddings
/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)
2022-07-15 03:10:35,779 - BERTopic - Reduced dimensionality


In [ ]:
new_topics = [np.argmax(prob) if max(prob) >= 0.01 else -1 for prob in probs]
topic_model.update_topics(stemmed_titles_j, new_topics)
documents = pd.DataFrame({"Document": stemmed_titles_j, "Topic": new_topics})
topic_model._update_topic_size(documents)

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart()

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,83265,-1_oper_specialist_analyst_technician
1,0,4744,0_mechan_driver_technician_engin
2,1,4103,1_market_research_assist_sale
3,2,3099,2_qualiti_assur_qualiti assur_control
4,3,3055,3_oper_machin_machin oper_press
5,4,2812,4_dispatch_coordin_contract_trader


In [ ]:
salarydot['topic'] = topics[0][:len(salarydot)]
payscale['topic'] = topics[0][len(salarydot):len(salarydot)+len(payscale)]
indeed['topic'] = topics[0][len(salarydot)+len(payscale):len(salarydot)+len(payscale)+len(indeed)]
glassdoor['topic'] = topics[0][len(salarydot)+len(payscale)+len(indeed):len(salarydot)+len(payscale)+len(indeed)+len(glassdoor)]
onet['topic'] = topics[0][len(salarydot)+len(payscale)+len(indeed)+len(glassdoor):len(salarydot)+len(payscale)+len(indeed)+len(glassdoor)+len(onet)]
onet_orig['topic'] = topics[0][len(salarydot)+len(payscale)+len(indeed)+len(glassdoor)+len(onet):]

In [ ]:
salarydot.to_csv('data/salarydot_upd.csv', index = False)
payscale.to_csv('data/payscale_upd.csv', index = False)
indeed.to_csv('data/indeed_upd.csv', index = False)
glassdoor.to_csv('data/glassdoor_upd.csv', index = False)
onet.to_csv('data/onet_upd.csv', index = False)
onet_orig.to_csv('data/onet_orig_upd.csv', index = False)

## Distinct match

In [ ]:
salarydot = pd.read_csv('data/salarydot_upd.csv')
payscale = pd.read_csv('data/payscale_upd.csv')
indeed = pd.read_csv('data/indeed_upd.csv')
glassdoor = pd.read_csv('data/glassdoor_upd.csv')
onet = pd.read_csv('data/onet_upd.csv')
onet_orig = pd.read_csv('data/onet_orig_upd.csv')  

In [ ]:
salarydot_indeed = salarydot.merge(indeed, how='inner', left_on='job_title_orig_clean', right_on='job_title_orig_clean', suffixes=['_salarydot', '_indeed'])
salarydot_glassdoor = salarydot.merge(glassdoor, how='inner', left_on='job_title_orig_clean', right_on='job_title_orig_clean', suffixes=['_salarydot', '_glassdoor'])
salarydot_payscale = salarydot.merge(payscale, how='inner', left_on='job_title_orig_clean', right_on='job_title_orig_clean', suffixes=['_salarydot', '_payscale'])
salarydot_onet = salarydot.merge(onet, how='inner', left_on='job_title_orig_clean', right_on='job_title_orig_clean', suffixes=['_salarydot', '_onet'])
salarydot_onet_orig = salarydot.merge(onet_orig, how='inner', left_on='job_title_orig_clean', right_on='job_title_orig_clean', suffixes=['_salarydot', '_onet'])

In [ ]:
salarydot_indeed_exact_num = len(salarydot_indeed)
salarydot_glassdoor_exact_num = len(salarydot_glassdoor)
salarydot_payscale_exact_num = len(salarydot_payscale)
salarydot_onet_exact_alt_num = len(salarydot_onet)
salarydot_onet_exact_num = len(salarydot_onet_orig)

In [ ]:
salarydot_indeed.to_csv('data/salarydot_indeed.csv')
salarydot_glassdoor.to_csv('data/salarydot_glassdoor.csv')
salarydot_payscale.to_csv('data/salarydot_payscale.csv')
salarydot_onet.to_csv('data/salarydot_onet.csv')
salarydot_onet_orig.to_csv('data/salarydot_onet_orig.csv')

## Distance calculation (Jaro-W) functions

In [ ]:
def jaro_score(a, b, method):
  
  if method == 'wordwise':
    a, b = a.split(), b.split()
    if len(a)>0 and len(b)>0:
      l = [max([jaro.jaro_winkler_metric(i,j) for j in b]) for i in a]
      l += [0] * int(abs(len(a) - len(b)))
      score = np.mean(l)
    else:
      score = 0
  else:
    score = jaro.jaro_winkler_metric(a, b)
  return score

In [ ]:
def calc_jaro_score(dataset1, dataset2, titles_col, first_m_col, method, print_every):
  jaro_dict = {}
  counter = 0
  for _, i in dataset1.iterrows():
    counter += 1
    for _, j in dataset2.iterrows():
      a, b = i[first_m_col], j[first_m_col]
      jaro_metric = jaro_score(a, b, method)

      if (i['level'] == j['level']) or (pd.isnull(i['level']) and pd.isnull(j['level'])):
        numeric_level_match = 0
      else:
        numeric_level_match = -1

      if (i['s_j'] == j['s_j']) or (pd.isnull(i['s_j']) and pd.isnull(j['s_j'])):
        word_level_match = 0
      else:
        word_level_match = -1
      
      if (i['m_e'] == j['m_e']) or (pd.isnull(i['m_e']) and pd.isnull(j['m_e'])):
        manager_engineer_match = 0
      else:
        manager_engineer_match = -1
      levels_metric = min(numeric_level_match, word_level_match)
      combined_metric = jaro_metric + 0.15*levels_metric + 0.15*manager_engineer_match
      if i[titles_col] in jaro_dict.keys():
        if jaro_dict[i[titles_col]][0] < combined_metric:
          jaro_dict[i[titles_col]] = [jaro_metric, numeric_level_match, word_level_match, manager_engineer_match, combined_metric, j[titles_col]]
        else:
          pass
      else:
        jaro_dict[i[titles_col]] = [jaro_metric, numeric_level_match, word_level_match, manager_engineer_match, combined_metric, j[titles_col]]
    # if counter%print_every==0:
    #   print(counter)
  jaro_df = pd.DataFrame.from_dict(jaro_dict, orient='index').reset_index()
  jaro_df.columns = ['title', 'jaro_score', 'numeric_level_match', 'word_level_match', 'manager_engineer_match', 'combined_score', 'matched_title']
  return jaro_df

In [ ]:
def calc_jaro_in_blocks(dataset1, dataset2, col_name, first_m_col, method, cond1, cond2):
  res = pd.DataFrame()
  u1 = list(set(dataset1[cond1]))
  u2 = list(set(dataset1[cond2]))

  for el1 in u1:
    for el2 in u2:
      # print(len(dataset1[(dataset1[cond1] == el1) & (dataset1[cond2] == el2)][col_name]), len(dataset2[(dataset2[cond1] == el1) & (dataset2[cond2] == el2)][col_name]))
      if len(dataset1[(dataset1[cond1] == el1) & (dataset1[cond2] == el2)][col_name]) and len(dataset2[(dataset2[cond1] == el1) & (dataset2[cond2] == el2)][col_name]):
        temp = calc_jaro_score(dataset1[(dataset1[cond1] == el1) & (dataset1[cond2] == el2)], dataset2[(dataset2[cond1] == el1) & (dataset2[cond2] == el2)], col_name, first_m_col, method, 100)
        res =  pd.concat([res, temp])
  return res

In [ ]:
def run_matching_round(orig_dataset1, orig_dataset2, prev_match_dataset, col_name, first_m_col, method, cond1, cond2, suffixes):
  round_dataset1 = orig_dataset1[~orig_dataset1['job_title_orig_clean'].isin(prev_match_dataset['job_title_orig_clean'])]
  round_dataset2 = orig_dataset2[~orig_dataset2['job_title_orig_clean'].isin(prev_match_dataset['job_title_orig_clean'])]

  matched_in_blocks = calc_jaro_in_blocks(round_dataset1, round_dataset2, col_name, first_m_col, method, cond1, cond2)
  matched_in_round = round_dataset1.merge(matched_in_blocks[matched_in_blocks.combined_score>0.82], how='inner', left_on=col_name, right_on='title', suffixes=suffixes)
  return matched_in_round.sort_values(by='combined_score', ascending=False)

## Round 1

In [ ]:
salarydot_indeed_matched_round1 = run_matching_round(salarydot, indeed, salarydot_indeed, 
                   col_name='job_title_orig_clean', first_m_col='job_title_clean_no_indicators', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_indeed'])
salarydot_indeed_matched_round1.to_csv('data/salarydot_indeed_matched_round1.csv')

In [ ]:
salarydot_glassdoor_matched_round1 = run_matching_round(salarydot, glassdoor, salarydot_glassdoor, 
                   col_name='job_title_orig_clean', first_m_col='job_title_clean_no_indicators', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_glassdoor'])
salarydot_glassdoor_matched_round1.to_csv('data/salarydot_glassdoor_matched_round1.csv')

In [ ]:
salarydot_payscale_matched_round1 = run_matching_round(salarydot, payscale, salarydot_payscale, 
                   col_name='job_title_orig_clean', first_m_col='job_title_clean_no_indicators', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_payscale'])
salarydot_payscale_matched_round1.to_csv('data/salarydot_payscale_matched_round1.csv')

In [ ]:
salarydot_onet_orig_matched_round1 = run_matching_round(salarydot, onet_orig, salarydot_onet_orig, 
                   col_name='job_title_orig_clean', first_m_col='job_title_clean_no_indicators', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_onet'])
salarydot_onet_orig_matched_round1.to_csv('data/salarydot_onet_orig_matched_round1.csv')

## Round 2

In [ ]:
salarydot_indeed_matched_round1 = pd.read_csv('data/salarydot_indeed_matched_round1.csv')
salarydot_glassdoor_matched_round1 = pd.read_csv('data/salarydot_glassdoor_matched_round1.csv')
salarydot_payscale_matched_round1 = pd.read_csv('data/salarydot_payscale_matched_round1.csv')
salarydot_onet_orig_matched_round1 = pd.read_csv('data/salarydot_onet_orig_matched_round1.csv')

In [ ]:
start1, start2 = salarydot[~salarydot['job_title_orig_clean'].isin(salarydot_indeed['job_title_orig_clean'])], indeed[~indeed['job_title_orig_clean'].isin(salarydot_indeed['job_title_orig_clean'])]
salarydot_indeed_matched_round2 = run_matching_round(start1, start2, salarydot_indeed_matched_round1[salarydot_indeed_matched_round1.combined_score>0.9], 
                   col_name='job_title_orig_clean', first_m_col='stemmed_title_j', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_indeed'])
salarydot_indeed_matched_round2.to_csv('data/salarydot_indeed_matched_round2.csv')

In [ ]:
start1, start2 = salarydot[~salarydot['job_title_orig_clean'].isin(salarydot_glassdoor['job_title_orig_clean'])], glassdoor[~glassdoor['job_title_orig_clean'].isin(salarydot_glassdoor['job_title_orig_clean'])]
salarydot_glassdoor_matched_round2 = run_matching_round(start1, start2, salarydot_glassdoor_matched_round1[salarydot_glassdoor_matched_round1.combined_score>0.9], 
                   col_name='job_title_orig_clean', first_m_col='stemmed_title_j', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_glassdoor'])
salarydot_glassdoor_matched_round2.to_csv('data/salarydot_glassdoor_matched_round2.csv')

In [ ]:
start1, start2 = salarydot[~salarydot['job_title_orig_clean'].isin(salarydot_payscale['job_title_orig_clean'])], payscale[~payscale['job_title_orig_clean'].isin(salarydot_payscale['job_title_orig_clean'])]
salarydot_payscale_matched_round2 = run_matching_round(start1, start2, salarydot_payscale_matched_round1, 
                   col_name='job_title_orig_clean', first_m_col='stemmed_title_j', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_payscale'])
salarydot_payscale_matched_round2.to_csv('data/salarydot_payscale_matched_round2.csv')

In [ ]:
start1, start2 = salarydot[~salarydot['job_title_orig_clean'].isin(salarydot_onet_orig['job_title_orig_clean'])], onet[~onet['job_title_orig_clean'].isin(salarydot_onet_orig['job_title_orig_clean'])]
salarydot_onet_matched_round2 = run_matching_round(start1, start2, salarydot_onet_orig_matched_round1, 
                   col_name='job_title_orig_clean', first_m_col='job_title_clean_no_indicators', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_onet'])
salarydot_onet_matched_round2.to_csv('data/salarydot_onet_matched_round2.csv')

## Round 3

In [ ]:
# changed levels condition to smth less strict
def calc_jaro_score(dataset1, dataset2, titles_col, first_m_col, method, print_every):
  jaro_dict = {}
  counter = 0
  for _, i in dataset1.iterrows():
    counter += 1
    for _, j in dataset2.iterrows():
      a, b = i[first_m_col], j[first_m_col]

      jaro_metric = jaro_score(a, b, method)

      if (i['level'] == j['level']) or pd.isnull(i['level']) or pd.isnull(j['level']):
        numeric_level_match = 0
      else:
        numeric_level_match = -1

      if (i['s_j'] == j['s_j']) or (pd.isnull(i['s_j']) and pd.isnull(j['s_j'])):
        word_level_match = 0
      else:
        word_level_match = -1
      
      if (i['m_e'] == j['m_e']) or (pd.isnull(i['m_e']) and pd.isnull(j['m_e'])):
        manager_engineer_match = 0
      else:
        manager_engineer_match = -1
      levels_metric = min(numeric_level_match, word_level_match)
      combined_metric = jaro_metric + 0.15*levels_metric + 0.15*manager_engineer_match
      if i[titles_col] in jaro_dict.keys():
        if jaro_dict[i[titles_col]][0] < combined_metric:
          jaro_dict[i[titles_col]] = [jaro_metric, numeric_level_match, word_level_match, manager_engineer_match, combined_metric, j[titles_col]]
        else:
          pass
      else:
        jaro_dict[i[titles_col]] = [jaro_metric, numeric_level_match, word_level_match, manager_engineer_match, combined_metric, j[titles_col]]
    # if counter%print_every==0:
    #   print(counter)
  jaro_df = pd.DataFrame.from_dict(jaro_dict, orient='index').reset_index()
  jaro_df.columns = ['title', 'jaro_score', 'numeric_level_match', 'word_level_match', 'manager_engineer_match', 'combined_score', 'matched_title']
  return jaro_df

In [ ]:
salarydot_indeed_matched_round2 = pd.read_csv('data/salarydot_indeed_matched_round2.csv')
salarydot_glassdoor_matched_round2 = pd.read_csv('data/salarydot_glassdoor_matched_round2.csv')
salarydot_payscale_matched_round2 = pd.read_csv('data/salarydot_payscale_matched_round2.csv')
# salarydot_onet_orig_matched_round1 = pd.read_csv('data/salarydot_onet_orig_matched_round1.csv')

In [ ]:
start1 = salarydot[(~salarydot['job_title_orig_clean'].isin(salarydot_indeed['job_title_orig_clean'])) & (~salarydot['job_title_orig_clean'].isin(salarydot_indeed_matched_round1[salarydot_indeed_matched_round1.combined_score>0.9]['job_title_orig_clean']))]
start2 = indeed[(~indeed['job_title_orig_clean'].isin(salarydot_indeed['job_title_orig_clean'])) & (~indeed['job_title_orig_clean'].isin(salarydot_indeed_matched_round1[salarydot_indeed_matched_round1.combined_score>0.9]['job_title_orig_clean']))]
salarydot_indeed_matched_round3 = run_matching_round(start1, start2, salarydot_indeed_matched_round2[salarydot_indeed_matched_round2.combined_score>0.9], 
                   col_name='job_title_orig_clean', first_m_col='job_title_clean_no_indicators', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_indeed'])
salarydot_indeed_matched_round3.to_csv('data/salarydot_indeed_matched_round3.csv')

In [ ]:
start1 = salarydot[(~salarydot['job_title_orig_clean'].isin(salarydot_glassdoor['job_title_orig_clean'])) & (~salarydot['job_title_orig_clean'].isin(salarydot_glassdoor_matched_round1[salarydot_glassdoor_matched_round1.combined_score>0.9]['job_title_orig_clean']))]
start2 = glassdoor[(~glassdoor['job_title_orig_clean'].isin(salarydot_glassdoor['job_title_orig_clean'])) & (~glassdoor['job_title_orig_clean'].isin(salarydot_glassdoor_matched_round1[salarydot_glassdoor_matched_round1.combined_score>0.9]['job_title_orig_clean']))]
salarydot_glassdoor_matched_round3 = run_matching_round(start1, start2, salarydot_glassdoor_matched_round2[salarydot_glassdoor_matched_round2.combined_score>0.9], 
                   col_name='job_title_orig_clean', first_m_col='job_title_clean_no_indicators', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_glassdoor'])
salarydot_glassdoor_matched_round3.to_csv('data/salarydot_glassdoor_matched_round3.csv')

In [ ]:
start1 = salarydot[(~salarydot['job_title_orig_clean'].isin(salarydot_payscale['job_title_orig_clean'])) & (~salarydot['job_title_orig_clean'].isin(salarydot_payscale_matched_round1[salarydot_payscale_matched_round1.combined_score>0.9]['job_title_orig_clean']))]
start2 = payscale[(~payscale['job_title_orig_clean'].isin(salarydot_payscale['job_title_orig_clean'])) & (~payscale['job_title_orig_clean'].isin(salarydot_payscale_matched_round1[salarydot_payscale_matched_round1.combined_score>0.9]['job_title_orig_clean']))]
salarydot_payscale_matched_round3 = run_matching_round(start1, start2, salarydot_payscale_matched_round2[salarydot_payscale_matched_round2.combined_score>0.9], 
                   col_name='job_title_orig_clean', first_m_col='job_title_clean_no_indicators', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_payscale'])
salarydot_payscale_matched_round3.to_csv('data/salarydot_payscale_matched_round3.csv')

## Round 4

In [ ]:
salarydot_indeed_matched_round3 = pd.read_csv('data/salarydot_indeed_matched_round3.csv')
salarydot_glassdoor_matched_round3 = pd.read_csv('data/salarydot_glassdoor_matched_round3.csv')
salarydot_payscale_matched_round3 = pd.read_csv('data/salarydot_payscale_matched_round3.csv')

In [ ]:
start1 = salarydot[(~salarydot['job_title_orig_clean'].isin(salarydot_indeed['job_title_orig_clean'])) & (~salarydot['job_title_orig_clean'].isin(salarydot_indeed_matched_round1[salarydot_indeed_matched_round1.combined_score>0.9]['job_title_orig_clean'])) & (~salarydot['job_title_orig_clean'].isin(salarydot_indeed_matched_round2[salarydot_indeed_matched_round2.combined_score>0.9]['job_title_orig_clean']))]
start2 = indeed[(~indeed['job_title_orig_clean'].isin(salarydot_indeed['job_title_orig_clean'])) & (~indeed['job_title_orig_clean'].isin(salarydot_indeed_matched_round1[salarydot_indeed_matched_round1.combined_score>0.9]['job_title_orig_clean'])) & (~indeed['job_title_orig_clean'].isin(salarydot_indeed_matched_round2[salarydot_indeed_matched_round2.combined_score>0.9]['job_title_orig_clean']))]
salarydot_indeed_matched_round4 = run_matching_round(start1, start2, salarydot_indeed_matched_round3[salarydot_indeed_matched_round3.combined_score>0.9], 
                   col_name='job_title_orig_clean', first_m_col='job_title_clean_no_indicators', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_indeed'])
salarydot_indeed_matched_round4.to_csv('data/salarydot_indeed_matched_round4.csv')

In [ ]:
start1 = salarydot[(~salarydot['job_title_orig_clean'].isin(salarydot_glassdoor['job_title_orig_clean'])) & (~salarydot['job_title_orig_clean'].isin(salarydot_glassdoor_matched_round1[salarydot_glassdoor_matched_round1.combined_score>0.9]['job_title_orig_clean'])) & (~salarydot['job_title_orig_clean'].isin(salarydot_glassdoor_matched_round2[salarydot_glassdoor_matched_round2.combined_score>0.9]['job_title_orig_clean']))]
start2 = glassdoor[(~glassdoor['job_title_orig_clean'].isin(salarydot_glassdoor['job_title_orig_clean'])) & (~glassdoor['job_title_orig_clean'].isin(salarydot_glassdoor_matched_round1[salarydot_glassdoor_matched_round1.combined_score>0.9]['job_title_orig_clean'])) & (~glassdoor['job_title_orig_clean'].isin(salarydot_glassdoor_matched_round2[salarydot_glassdoor_matched_round2.combined_score>0.9]['job_title_orig_clean']))]
salarydot_glassdoor_matched_round4 = run_matching_round(start1, start2, salarydot_glassdoor_matched_round3[salarydot_glassdoor_matched_round3.combined_score>0.9], 
                   col_name='job_title_orig_clean', first_m_col='job_title_clean_no_indicators', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_glassdoor'])
salarydot_glassdoor_matched_round4.to_csv('data/salarydot_glassdoor_matched_round4.csv')

In [ ]:
start1 = salarydot[(~salarydot['job_title_orig_clean'].isin(salarydot_payscale['job_title_orig_clean'])) & (~salarydot['job_title_orig_clean'].isin(salarydot_payscale_matched_round1[salarydot_payscale_matched_round1.combined_score>0.9]['job_title_orig_clean'])) & (~salarydot['job_title_orig_clean'].isin(salarydot_payscale_matched_round2[salarydot_payscale_matched_round2.combined_score>0.9]['job_title_orig_clean']))]
start2 = payscale[(~payscale['job_title_orig_clean'].isin(salarydot_payscale['job_title_orig_clean'])) & (~payscale['job_title_orig_clean'].isin(salarydot_payscale_matched_round1[salarydot_payscale_matched_round1.combined_score>0.9]['job_title_orig_clean'])) & (~payscale['job_title_orig_clean'].isin(salarydot_payscale_matched_round2[salarydot_payscale_matched_round2.combined_score>0.9]['job_title_orig_clean']))]
salarydot_payscale_matched_round4 = run_matching_round(start1, start2, salarydot_payscale_matched_round3[salarydot_payscale_matched_round3.combined_score>0.9], 
                   col_name='job_title_orig_clean', first_m_col='job_title_clean_no_indicators', 
                   method='wordwise', cond1='topic', cond2='first_letter', suffixes=['_salarydot', '_payscale'])
salarydot_payscale_matched_round4.to_csv('data/salarydot_payscale_matched_round4.csv')

## Combine rounds

In [ ]:
salarydot_indeed_matched_round1 = pd.read_csv('data/salarydot_indeed_matched_round1.csv')
salarydot_indeed_matched_round2 = pd.read_csv('data/salarydot_indeed_matched_round2.csv')
salarydot_indeed_matched_round3 = pd.read_csv('data/salarydot_indeed_matched_round3.csv')
salarydot_indeed_matched_round4 = pd.read_csv('data/salarydot_indeed_matched_round4.csv')
salarydot_glassdoor_matched_round1 = pd.read_csv('data/salarydot_glassdoor_matched_round1.csv')
salarydot_glassdoor_matched_round2 = pd.read_csv('data/salarydot_glassdoor_matched_round2.csv')
salarydot_glassdoor_matched_round3 = pd.read_csv('data/salarydot_glassdoor_matched_round3.csv')
salarydot_glassdoor_matched_round4 = pd.read_csv('data/salarydot_glassdoor_matched_round4.csv')
salarydot_payscale_matched_round1 = pd.read_csv('data/salarydot_payscale_matched_round1.csv')
salarydot_payscale_matched_round2 = pd.read_csv('data/salarydot_payscale_matched_round2.csv')
salarydot_payscale_matched_round3 = pd.read_csv('data/salarydot_payscale_matched_round3.csv')
salarydot_payscale_matched_round4 = pd.read_csv('data/salarydot_payscale_matched_round4.csv')

In [ ]:
salarydot_indeed_matched_round1['round'], salarydot_indeed_matched_round2['round'], salarydot_indeed_matched_round3['round'], salarydot_indeed_matched_round4['round'] = 1,2,3,4
salarydot_glassdoor_matched_round1['round'], salarydot_glassdoor_matched_round2['round'], salarydot_glassdoor_matched_round3['round'], salarydot_glassdoor_matched_round4['round'] = 1,2,3,4
salarydot_payscale_matched_round1['round'], salarydot_payscale_matched_round2['round'], salarydot_payscale_matched_round3['round'], salarydot_payscale_matched_round4['round'] = 1,2,3,4

In [ ]:
salarydot_indeed_allrounds = pd.concat([salarydot_indeed_matched_round1, salarydot_indeed_matched_round2, salarydot_indeed_matched_round3, salarydot_indeed_matched_round4], ignore_index=True, sort=False).drop_duplicates(subset=['job_title_orig_clean'], keep='first')
salarydot_glassdoor_allrounds = pd.concat([salarydot_glassdoor_matched_round1, salarydot_glassdoor_matched_round2, salarydot_glassdoor_matched_round3, salarydot_glassdoor_matched_round4], ignore_index=True, sort=False).drop_duplicates(subset=['job_title_orig_clean'], keep='first')
salarydot_payscale_allrounds = pd.concat([salarydot_payscale_matched_round1, salarydot_payscale_matched_round2, salarydot_payscale_matched_round3, salarydot_payscale_matched_round4], ignore_index=True, sort=False).drop_duplicates(subset=['job_title_orig_clean'], keep='first')


In [ ]:
salarydot_indeed_matched_full = salarydot_indeed_allrounds.merge(indeed, how='left', left_on='matched_title', right_on='job_title_orig_clean', suffixes=['_salarydot', '_indeed'])
salarydot_glassdoor_matched_full = salarydot_glassdoor_allrounds.merge(glassdoor, how='left', left_on='matched_title', right_on='job_title_orig_clean', suffixes=['_salarydot', '_glassdoor'])
salarydot_payscale_matched_full = salarydot_payscale_allrounds.merge(payscale, how='left', left_on='matched_title', right_on='job_title_orig_clean', suffixes=['_salarydot', '_payscale'])

In [ ]:
salarydot_indeed_matched_full.reset_index(drop=True).to_csv('salarydot_indeed_matched_full.csv')
salarydot_glassdoor_matched_full.reset_index(drop=True).to_csv('salarydot_glassdoor_matched_full.csv')
salarydot_payscale_matched_full.reset_index(drop=True).to_csv('salarydot_payscale_matched_full.csv')

In [ ]:
salarydot_indeed['round'] = 0
salarydot_glassdoor['round'] = 0
salarydot_payscale['round'] = 0

In [ ]:
indeed_final = pd.concat([salarydot_indeed, salarydot_indeed_matched_full], ignore_index=True, sort=False)
indeed_final = indeed_final.drop(columns=['Unnamed: 0'])

In [ ]:
glassdoor_final = pd.concat([salarydot_glassdoor, salarydot_glassdoor_matched_full], ignore_index=True, sort=False)
glassdoor_final = glassdoor_final.drop(columns=['Unnamed: 0'])

In [ ]:
payscale_final = pd.concat([salarydot_payscale, salarydot_payscale_matched_full], ignore_index=True, sort=False)
payscale_final = payscale_final.drop(columns=['Unnamed: 0'])

In [ ]:
print(len(indeed_final), len(glassdoor_final), len(payscale_final))

9505 1464 5477


## Combining w onet

In [ ]:
# payscale_final = pd.read_csv('data/payscale_final.csv')
# indeed_final = pd.read_csv('data/indeed_final.csv')
# glassdoor_final = pd.read_csv('data/glassdoor_final.csv')
salarydot_onet = pd.read_csv('data/salarydot_onet.csv') 
salarydot_onet_orig = pd.read_csv('data/salarydot_onet_orig.csv') 
salarydot_onet_matched_round1 = pd.read_csv('data/salarydot_onet_matched_round1.csv')
salarydot_onet_matched_round2 = pd.read_csv('data/salarydot_onet_matched_round2.csv')

In [ ]:
len(set(salarydot_onet_matched_round1['job_title_orig_clean']))

2273

In [ ]:
len(set(salarydot_onet_matched_round2['job_title_orig_clean']))

3844

In [ ]:
onet_exact = salarydot_onet[['job_title_orig_clean', 'alternate_title', 'title', 'soc_code']]
onet_orig_exact = salarydot_onet_orig[['job_title_orig_clean', 'title', 'soc_code']]

In [ ]:
pd.concat([onet_exact, onet_orig_exact], ignore_index=True, sort=False)

In [ ]:
temp = salarydot_onet_matched_round1.merge(onet_orig, how='left', left_on='matched_title', right_on='job_title_orig_clean', suffixes=['_salarydot', '_onet'])
temp = salarydot_onet_matched_round1.merge(onet, how='left', left_on='matched_title', right_on='job_title_orig_clean', suffixes=['_salarydot', '_onet'])
temp = temp.drop_duplicates(subset=['job_title_orig_clean_salarydot'], keep='first')

In [ ]:
onet_matched = temp[['job_title_orig_clean_salarydot', 'alternate_title', 'title_onet', 'soc_code']]
onet_orig_matched = temp[['job_title_orig_clean_salarydot', 'title_onet', 'soc_code']]

In [ ]:
onet_orig_exact.columns = ['job_title_orig_clean', 'title_onet', 'soc_code']
onet_exact.columns = ['job_title_orig_clean', 'alternate_title', 'title_onet', 'soc_code']
onet_matched.columns = ['job_title_orig_clean', 'alternate_title', 'title_onet', 'soc_code']
onet_orig_matched.columns = ['job_title_orig_clean', 'title_onet', 'soc_code']

In [ ]:
onet_orig_exact['round'], onet_exact['round'], onet_orig_matched['round'], onet_matched['round'] = 0,0,1,2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
onet_final = pd.concat([onet_orig_exact, onet_exact, onet_orig_matched, onet_matched], ignore_index=True, sort=False)

In [ ]:
onet_final = onet_final.drop_duplicates(subset=['job_title_orig_clean', 'title_onet', 'soc_code'])

In [ ]:
len(onet_final)

4237

In [ ]:
onet_final.to_csv('data/onet_matched_wsalarydot.csv')

In [ ]:
payscale_final_w_onet = payscale_final.merge(onet_final, how='left', left_on = 'job_title_orig_clean_salarydot', right_on= 'job_title_orig_clean', suffixes=['', '_onet'])
indeed_final_w_onet = indeed_final.merge(onet_final, how='left', left_on = 'job_title_orig_clean_salarydot', right_on= 'job_title_orig_clean', suffixes=['', '_onet'])
glassdoor_final_w_onet = glassdoor_final.merge(onet_final, how='left', left_on = 'job_title_orig_clean_salarydot', right_on= 'job_title_orig_clean', suffixes=['', '_onet'])

In [ ]:
payscale_final_w_onet.to_csv('data/payscale_final_w_onet.csv')
indeed_final_w_onet.to_csv('data/indeed_final_w_onet.csv')
glassdoor_final_w_onet.to_csv('data/glassdoor_final_w_onet.csv')

## Quality check

In [ ]:
salarydot_120 = pd.read_excel('data/salarydot_120.xlsx')

In [ ]:
salarydot_120['job_title_clean'] = [i.replace('&amp;', 'and').replace('&#39;s', '') for i in salarydot_120['JobTitle'] if not i.isdigit()]
salarydot_120 = preprocess(salarydot_120, 'job_title_clean')

In [ ]:
salarydot_120_merged = salarydot_120.merge(onet_final, how='left', left_on = 'job_title_orig_clean', right_on= 'job_title_orig_clean', suffixes=['', '_onet'])

In [ ]:
salarydot_120_merged[['JobTitle', '2018 SOC CODE', 'soc_code', '2018 SOC OCCUPATION', 'alternate_title', 'title_onet',
       'TXNYJobLevelCode', 'TXNYJobFamilyName']][~salarydot_120_merged.soc_code.isna()].to_csv('data/quality_check_120.csv')

In [ ]:
salarydot_120_merged.shape

(312, 28)

## Charts

In [ ]:
def mm(graph):
  graphbytes = graph.encode("ascii")
  base64_bytes = base64.b64encode(graphbytes)
  base64_string = base64_bytes.decode("ascii")
  display(Image(url="https://mermaid.ink/img/" + base64_string))

var1 = 0.9 
var2 = 0.82 

mm(f"""graph TD
    A["Original data"] 
    -->|Preprocessing| B("Round 0  <br/> (S: {len(salarydot)} I: {len(indeed)} P: {len(payscale)} G: {len(glassdoor)})")
    -->|No match| C("Round 1 <br/> (I: {len(indeed) - salarydot_indeed_exact_num} P: {len(payscale) - salarydot_payscale_exact_num} G: {len(glassdoor) - salarydot_glassdoor_exact_num})")
    -->|Combined score < {var1}| D("Round 2 <br/>  (I: {len(indeed) - salarydot_indeed_exact_num - len(set(salarydot_indeed_matched_round1.matched_title))} 
    P: {len(payscale) - salarydot_payscale_exact_num - len(set(salarydot_payscale_matched_round1.matched_title))} 
    G: {len(glassdoor) - salarydot_glassdoor_exact_num - len(set(salarydot_glassdoor_matched_round1.matched_title))})")
    -->|Combined score < {var1}| F("Round 3 <br/>  (I: {len(indeed) - salarydot_indeed_exact_num - len(set(salarydot_indeed_matched_round1.matched_title)) - len(set(salarydot_indeed_matched_round2.matched_title))} 
    P: {len(payscale) - salarydot_payscale_exact_num - len(set(salarydot_payscale_matched_round1.matched_title)) - len(set(salarydot_payscale_matched_round2.matched_title))} 
    G: {len(glassdoor) - salarydot_glassdoor_exact_num - len(set(salarydot_glassdoor_matched_round1.matched_title)) - len(set(salarydot_glassdoor_matched_round2.matched_title))})")
    -->|Combined score < {var1}| G("Round 4 <br/> (I: {len(indeed) - salarydot_indeed_exact_num - len(set(salarydot_indeed_matched_round1.matched_title)) - len(set(salarydot_indeed_matched_round2.matched_title)) - len(set(salarydot_indeed_matched_round3.matched_title))} 
    P: {len(payscale) - salarydot_payscale_exact_num - len(set(salarydot_payscale_matched_round1.matched_title)) - len(set(salarydot_payscale_matched_round2.matched_title)) - len(set(salarydot_payscale_matched_round3.matched_title))} 
    G: {len(glassdoor) - salarydot_glassdoor_exact_num - len(set(salarydot_glassdoor_matched_round1.matched_title)) - len(set(salarydot_glassdoor_matched_round2.matched_title)) - len(set(salarydot_glassdoor_matched_round3.matched_title))+55})")
    -->|Combined score < {var2}| H["Not matched <br/> (I: {len(indeed) - salarydot_indeed_exact_num - len(set(salarydot_indeed_matched_round1.matched_title)) - len(set(salarydot_indeed_matched_round2.matched_title)) - len(set(salarydot_indeed_matched_round3.matched_title)) - len(set(salarydot_indeed_matched_round4.matched_title))} 
    P: {len(payscale) - salarydot_payscale_exact_num - len(set(salarydot_payscale_matched_round1.matched_title)) - len(set(salarydot_payscale_matched_round2.matched_title)) - len(set(salarydot_payscale_matched_round3.matched_title)) - len(set(salarydot_payscale_matched_round4.matched_title))} 
    G: {len(glassdoor) - salarydot_glassdoor_exact_num - len(set(salarydot_glassdoor_matched_round1.matched_title)) - len(set(salarydot_glassdoor_matched_round2.matched_title)) - len(set(salarydot_glassdoor_matched_round3.matched_title)) + 50 })"]
    B --> |Exact match| I("Matched <br/> (I: {salarydot_indeed_exact_num} P: {salarydot_payscale_exact_num} G: {salarydot_glassdoor_exact_num})")
    C --> |> {var1}| J("Matched <br/> (I: {len(set(salarydot_indeed_matched_round1.matched_title))} P: {len(set(salarydot_payscale_matched_round1.matched_title))} G: {len(set(salarydot_glassdoor_matched_round1.matched_title))})")
    D --> |> {var1}| K("Matched <br/> (I: {len(set(salarydot_indeed_matched_round2.matched_title))} P: {len(set(salarydot_payscale_matched_round2.matched_title))} G: {len(set(salarydot_glassdoor_matched_round2.matched_title))})")
    F --> |> {var1}| L("Matched <br/> (I: {len(set(salarydot_indeed_matched_round3.matched_title))} P: {len(set(salarydot_payscale_matched_round3.matched_title))} G: {len(set(salarydot_glassdoor_matched_round3.matched_title))})")
    G --> |> {var2}| M("Matched <br/> (I: {len(set(salarydot_indeed_matched_round4.matched_title))} P: {len(set(salarydot_payscale_matched_round4.matched_title))} G: {len(set(salarydot_glassdoor_matched_round4.matched_title))})")
    style I fill:#82dfb0;
    style J fill:#82dfb0;
    style K fill:#82dfb0;
    style L fill:#82dfb0;
    style M fill:#82dfb0;
    style H fill:#fa8072;
""")